In [1]:
from IPython.display import display, Markdown, HTML
from tbh.paths import REPO_ROOT_PATH, DATA_FOLDER
# analysis_path = REPO_ROOT_PATH / "remote_cluster" / "outputs" / "49574599_25sc_revised_se" / "task_1"
analysis_path = REPO_ROOT_PATH / "remote_cluster" / "outputs" / "49854135_generalised_trans" / "task_1"

# analysis_path = REPO_ROOT_PATH / "notebooks" / "test_outputs" / "25scenarios_18dec"

In [2]:
import tbh.plotting as pl
import tbh.runner_tools as rt

import pandas as pd
import arviz as az
from matplotlib import pyplot as plt 
plt.style.use("ggplot")


In [3]:
intervention_scenarios = [sc.sc_id for sc in rt.SCENARIOS]
all_scenarios = ['baseline'] + intervention_scenarios
unc_dfs = {
    sc: pd.read_parquet(analysis_path / f"uncertainty_df_{sc}.parquet") for sc in all_scenarios
}
diff_outputs_dfs = {
    sc: pd.read_parquet(analysis_path / f"diff_quantiles_df_ref_baseline_{sc}.parquet") for sc in intervention_scenarios
}
idata = az.from_netcdf(analysis_path / "idata.nc")

In [4]:
import yaml

with open(analysis_path / "details.yaml" , "r") as f:
    docs = list(yaml.safe_load_all(f))

model_config = docs[1]
analysis_config = docs[2]

In [5]:
from tbh.model import get_tb_model
from estival.model import BayesianCompartmentalModel

params, priors, tv_params = rt.get_parameters_and_priors()

model = get_tb_model(model_config, tv_params)
bcm = BayesianCompartmentalModel(model, params, priors, rt.targets)

# Background and Introduction
## Aims and simulated population
This modelling analysis aims to explore and compare various scenarios of screening for TB and TBI in Kiribati. It focuses on simulating what the next phase of PEARL screening could potentially look like under different diagnostic approaches and considering different screening rates.
For this purpose, the simulated population is the population of South Tarawa excluding that already screened by the end of 2025. Thus, the model is designed to capture the part of South Tarawa extending from Nanikai to Bonriki.

We will use the data collected to date through the PEARL study to estimate TB and TBI prevalence in the population that has already been screened and will assume that similar epidemiological patterns apply to the population yet to be screened. These prevalence estimates will serve as calibration targets for the modelling exercise, with certain parameters automatically adjusted to ensure that the model reproduces the observed estimates.

In this report, we will use the following abbreviations for screening tools: SSx (symptom screening), CXR (computer-aided chest X-ray), Xpert (Xpert MTB/RIF Ultra), PLTS (Pluslife Tongue Swab), and TST (tuberculin skin test). 

## Modelled scenarios
The table below summarises the screening scenarios evaluated in this analysis. Scenario 0 represents a base case with no active case finding, included for comparison. Scenarios 1–25 progressively increase screening rate by expanding target coverage and modifying screening components (e.g. CXR, Xpert, and TST). Each scenario is defined by the proportion of eligible individuals that would be effectively screened with the PEARL activities in 2026. For all scenarios, we assume a TB treatment success rate of 95% and a TPT completion rate of 70%.

In [6]:
rows = [
    {
        "Sc.": "0",
        "Strategy / Coverage": "No Screening",
        "Coverage": "NA",
        "Description": "No active case finding or TBI screening",
    }
]
last_sc_name = ""
for sc in rt.SCENARIOS:
    
    desc = sc.description
    coverage = (
        sc.scr_prgs[0].total_coverage_perc
        if sc.scr_prgs else 0
    )

    sc_name = sc.sc_name.split(". ")[1]

    written_desc = desc if sc_name.split("/")[0] != last_sc_name.split("/")[0] else ""
    

    row = {
        "Sc.": sc.sc_name.split(".")[0],
        "Strategy / Coverage": sc_name,
        "Coverage": f"{round(coverage)}%",
        "Description": written_desc,
    }

    rows.append(row)

    last_sc_name = sc_name

df = pd.DataFrame(rows)
display(Markdown(df.to_markdown(index=False)))

|   Sc. | Strategy / Coverage    | Coverage   | Description                                                                                          |
|------:|:-----------------------|:-----------|:-----------------------------------------------------------------------------------------------------|
|     0 | No Screening           | NA         | No active case finding or TBI screening                                                              |
|     1 | PEARL / Low            | 55%        | PEARL algorithm: CXR-Xpert for 35% of 10+yr olds (CXR only for other 65%), SSX and TST for 3+yr olds |
|     2 | PEARL / Med            | 65%        |                                                                                                      |
|     3 | PEARL / High           | 75%        |                                                                                                      |
|     4 | PEARL / Vhigh          | 85%        |                                                                                                      |
|     5 | PEARL / Max            | 95%        |                                                                                                      |
|     6 | CXR-TST / Low          | 55%        | Dropping Xpert: CXR for 10+yr olds, SSx and TST for 3+yr olds                                        |
|     7 | CXR-TST / Med          | 65%        |                                                                                                      |
|     8 | CXR-TST / High         | 75%        |                                                                                                      |
|     9 | CXR-TST / Vhigh        | 85%        |                                                                                                      |
|    10 | CXR-TST / Max          | 95%        |                                                                                                      |
|    11 | CXR-TST 10+yrs / Low   | 55%        | Dropping Xpert and stop screening <10yrs: CXR, SSx and TST for 10+yr olds only                       |
|    12 | CXR-TST 10+yrs / Med   | 65%        |                                                                                                      |
|    13 | CXR-TST 10+yrs / High  | 75%        |                                                                                                      |
|    14 | CXR-TST 10+yrs / Vhigh | 85%        |                                                                                                      |
|    15 | CXR-TST 10+yrs / Max   | 95%        |                                                                                                      |
|    16 | CXR 10+yrs / Low       | 55%        | Dropping Xpert and TST, stop screening <10yrs: CXR and SSx for 10+yr olds only                       |
|    17 | CXR 10+yrs / Med       | 65%        |                                                                                                      |
|    18 | CXR 10+yrs / High      | 75%        |                                                                                                      |
|    19 | CXR 10+yrs / Vhigh     | 85%        |                                                                                                      |
|    20 | CXR 10+yrs / Max       | 95%        |                                                                                                      |
|    21 | CXR-PLTS-TST / Low     | 55%        | Use Tongue Swabs (PLTS): CXR and PLTS for 10+yr olds, SSx and TST for 3+yr olds                      |
|    22 | CXR-PLTS-TST / Med     | 65%        |                                                                                                      |
|    23 | CXR-PLTS-TST / High    | 75%        |                                                                                                      |
|    24 | CXR-PLTS-TST / Vhigh   | 85%        |                                                                                                      |
|    25 | CXR-PLTS-TST / Max     | 95%        |                                                                                                      |

# Model calibration and modelled baseline epidemic
## Model fits to data
@fig-multifit shows the model fits to calibration targets derived from data collected during the intervention. The black dots represent the observed quantities, whereas model estimates are shown in blue (solid line: median; dark shade: interquartile range; light shade: 95% CI).

In [7]:
#| fig-pos: "H" 
#| label: fig-multifit
#| fig-cap: "Model fits to observations. The black dots represent the observed quantities, whereas model estimates are shown in blue (solid line: median; dark shade: interquartile range; light shade: 95% CI)."
fig = pl.plot_all_model_fits(unc_dfs['baseline'], bcm, n_col=2, excluded_outputs=[o for o in bcm.targets if (o.startswith("tst_pos") and o != "tst_posXage_18+_perc")])
display(fig)
plt.close(fig)

/Users/rrag0004/Documents/Code/tb_hierarchical/code/tbh/plotting.py:312: UserWarning:

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.



<Figure size 3000x3240 with 5 Axes>

## Parameters' posterior distributions
@fig-posteriors shows a comparison between the prior and posterior distributions of the calibrated model parameters. The prior distributions, shown in grey, reflect our knowledge about the parameter values **before** running the analysis and before considering the data used for calibration. Prior distributions are often chosen to be non-informative when there is very little evidence available to inform the parameter value. The posterior distributions, displayed in red, show the parameter values accepted during model calibration. These are the values found to be 'plausible', or appropriate to produce reasonable fits to the observations.

In [8]:
#| fig-pos: "H" 
#| label: fig-posteriors
#| fig-cap: "Prior and posterior distributions of calibrated parameters. Prior distributions are shown in grey, and posterior distributions in red."
fig = pl.plot_post_prior_comparison(idata, analysis_config['burn_in'], req_vars=list(bcm.priors.keys()),
                              priors=list(bcm.priors.values()))
display(fig)
plt.close(fig)

<Figure size 7590x4830 with 16 Axes>

## Model outputs for non-fitted epidemic indicators
### Past epidemic trajectories

In [9]:
# selected_outputs = ['tb_incidence_per100k', 'tb_mortality_per100k', 'viable_tbi_prevalence_perc', 'passive_detection_rate_clin']
selected_outputs = ['tb_incidence_per100k', 'tb_mortality_per100k', 'passive_detection_rate_clin']

for output in selected_outputs:
    out_name = output if output not in pl.title_lookup else pl.title_lookup[output]
    display(Markdown(f"**{out_name}**"))
    fig, ax = plt.subplots(figsize=(5, 3))
    x_min = 1950 if output == "passive_detection_rate_clin" else 1990
    pl.plot_model_fit_with_uncertainty(ax, unc_dfs['baseline'], output, bcm, x_lim=(x_min, 2025), ylab_fontsize=10)
    display(fig)
    plt.close(fig)


**TB incidence (/100k)**

<Figure size 1500x900 with 1 Axes>

**TB mortality (/100k)**

<Figure size 1500x900 with 1 Axes>

**Passive detec. rate (/y), clinical TB**

<Figure size 1500x900 with 1 Axes>

### Estimated age-specific TST positivity rates

In [10]:
#| fig-pos: "H" 
#| label: fig-age_tbi
#| fig-cap: "Age-specific TST positivity rate (observed and modelled). Red crosses indicate the measured TST positivity rate (%). Blue boxes represent the model estimates (median, interquartile range, 95% CI)."
fig = pl.plot_age_spec_tbi_prev(unc_dfs['baseline'], bcm)
display(fig)
plt.close(fig)

<Figure size 2400x1500 with 1 Axes>

# Estimated cumulative impact of interventions on TB disease and mortality

## Cumulative outputs over 2026-2035

In [11]:
def add_scenario_annotations(ax, x_sep_positions, labels):

    # Get top of y-axis for label placement
    ymax = ax.get_ylim()[1]

    prev = 0
    for i, label in enumerate(labels):
        # Vertical line
        vline_pos = x_sep_positions[i]
        if i < len(x_sep_positions):
            ax.axvline(x=vline_pos, linestyle='--', linewidth=0.5, color='black')

        # # Text slightly above the plot
        ax.text(
            prev + (vline_pos - prev) / 2,
            ymax * 1.01,       # a bit above the top
            label,
            ha='center',
            va='bottom',
            rotation=0,
            clip_on=False
        )

        prev = vline_pos

In [12]:
for output in ['cum_tb_incidence', 'cum_tb_mortality']:
    
    fig, ax = plt.subplots(1, 1, figsize=(0.7*len(all_scenarios), 5.))
    pl.plot_final_size_compare(ax,unc_dfs, output, all_scenarios, end_year=2035)
    plt.setp(ax.get_xticklabels(), rotation=30, ha='right')

    add_scenario_annotations(ax, [1.5, 6.5, 11.5, 16.5, 21.5, 26.5] , ["", "PEARL", "CXR-TST", "CXR-TST 10+yrs", "CXR 10+yrs", "CXR-PLTS-TST"])

    display(fig)
    plt.close(fig)

<Figure size 5460x1500 with 1 Axes>

<Figure size 5460x1500 with 1 Axes>

## TB episodes and deaths averted over 2026-2035 (ref. no intervention)

In [13]:
for output in ["TB_averted", "TB_averted_relative", "deaths_averted", "deaths_averted_relative"]:
    fig, ax = plt.subplots(1, 1, figsize=(0.7*len(intervention_scenarios), 5.))
    pl.plot_diff_outputs(ax, diff_outputs_dfs, output, intervention_scenarios)
    plt.setp(ax.get_xticklabels(), rotation=30, ha='right')

    add_scenario_annotations(ax, [5.5, 10.5, 15.5, 20.5, 25.5] , ["PEARL", "CXR-TST", "CXR-TST 10+yrs", "CXR 10+yrs", "CXR-PLTS-TST"])


    display(fig)
    plt.close(fig)

<Figure size 5250x1500 with 1 Axes>

<Figure size 5250x1500 with 1 Axes>

<Figure size 5250x1500 with 1 Axes>

<Figure size 5250x1500 with 1 Axes>

# Technical details
## Model structure


![Model structure. Not shown: age-stratification; natural mortality (all compartments); TB mortality (clinical TB compartments); self-recovery (subclinical TB compartments); reinfection from 'Contained', 'Cleared' and 'Recovered' compartments transitions back to 'Incipient'.](tb_model.png){#fig:tb_model width=100%}

## Model parameters

In [14]:
#| tab-params: "H" 
#| label: tab-params
#| tab-cap: "Model parameters"
params_file_path = DATA_FOLDER / "parameters.xlsx"
param_df = pd.read_excel(params_file_path, sheet_name="constant")
param_df["value_or_prior"] = param_df.apply(
    lambda row: f"{row['distribution']} ({row['distri_param1']}, {row['distri_param2']})"
    if pd.notna(row['distribution'])
    else str(row['value']),
    axis=1
)
param_df = param_df.rename(columns={"full_text": "definition"})
param_df = param_df.fillna("")
md_table = param_df[["parameter", "definition", "value_or_prior", "unit"]].to_markdown(index=False)  # index=False to skip the row numbers
display(Markdown(md_table))

| parameter                    | definition                                                                              | value_or_prior           | unit   |
|:-----------------------------|:----------------------------------------------------------------------------------------|:-------------------------|:-------|
| raw_transmission_rate        | Effective rate of transmission (before adjusting for infectiousness)                    | uniform (1e-05, 0.001)   |        |
| mixing_factor_cc             | Child-child mixing, relative to adult-adult mixing                                      | 1.0                      |        |
| mixing_factor_ca             | Child-adult mixing, relative to adult-adult mixing                                      | 1.0                      |        |
| rel_sus_contained            | Rel. risk of reinfection from 'contained' compartment (ref. 'mtb-naïve')                | uniform (0.2, 0.5)       |        |
| rel_sus_cleared              | Rel. risk of reinfection from 'cleared' compartment (ref. 'mtb-naïve')                  | uniform (0.5, 1.0)       |        |
| rel_sus_children             | Rel. susceptibility to infection for under 15 year-old individuals (ref. 15 and over)   | 0.5                      |        |
| rel_infectiousness_subclin   | Rel. infectiousness of subclinical TB (ref. clinical TB)                                | 0.5                      |        |
| rel_infectiousness_lowinf    | Rel. infectiousness of 'less infectious' TB (ref. 'more infectious' TB)                 | 0.4                      |        |
| progression_rate_age0        | Rate of progression from 'incipient' to TB disease (age 0-4)                            | 2.4                      | /year  |
| progression_rate_age5        | Rate of progression from 'incipient' to TB disease (age 5-14)                           | 2.0                      | /year  |
| progression_rate_age15       | Rate of progression from 'incipient' to TB disease (age 15 and over)                    | 0.1                      | /year  |
| progression_prop_infectious  | Proportion of incident TB that is 'more infectious'                                     | 0.5                      | /year  |
| containment_rate_age0        | Rate of transition from 'incipient' to 'contained' (age 0-4)                            | 4.4                      | /year  |
| containment_rate_age5        | Rate of transition from 'incipient' to 'contained' (age 5-14)                           | 4.4                      | /year  |
| containment_rate_age15       | Rate of transition from 'incipient' to 'contained' (age 15 and over)                    | 2.0                      | /year  |
| breakdown_rate               | Rate of transition from 'contained' to 'incipient' (all ages)                           | uniform (0.01, 1.0)      | /year  |
| clearance_rate               | Rate of transition from 'contained' to 'cleared' (all ages)                             | uniform (0.01, 0.1)      | /year  |
| clinical_progression_rate    | Rate of progression from subclinical to clinical TB                                     | uniform (0.5, 5.0)       | /year  |
| clinical_regression_rate     | Rate of transition from clinical to subclinical TB                                      | 1.0                      | /year  |
| infectiousness_gain_rate     | Rate of progression from 'less infectious' to 'more infectious' TB                      | uniform (0.5, 5.0)       | /year  |
| infectiousness_loss_rate     | Rate of transition from 'more infectious' to 'less infectious' TB                       | 1.0                      | /year  |
| tb_mortality_rate_inf        | Rate of TB mortality for 'more infectious' clinical disease                             | 0.389                    | /year  |
| tb_mortality_rate_lowinf     | Rate of TB mortality for 'less infectious' clinical disease                             | 0.025                    | /year  |
| self_recovery_rate           | Rate of self-recovery for subclinical TB                                                | 0.4                      | /year  |
| recent_detection_rate        | Annual rate of TB detection in 2020                                                     | uniform (0.1, 5.0)       | /year  |
| tx_duration                  | Average TB treatment duration                                                           | 0.5                      | year   |
| pct_neg_tx_death             | Percentage of deaths among negative TB treatment outcomes                               | 40.0                     | %      |
| prev_se_incipient            | Probability of TST positivity for 'incipient' category                                  | 0.75                     |        |
| prev_se_contained            | Probability of TST positivity for 'contained' category                                  | 0.75                     |        |
| prev_se_cleared              | Probability of TST positivity for 'cleared' category                                    | uniform (0.2, 0.5)       |        |
| prev_se_subclin_lowinf_pearl | Probability PEARL-positive for 'subclinical' and 'less infectious' category             | 1.0                      |        |
| prev_se_clin_lowinf_pearl    | Probability PEARL-positive for 'clinical' and 'less infectious' category                | 1.0                      |        |
| prev_se_subclin_inf_pearl    | Probability PEARL-positive for 'subclinical' and 'more infectious' category             | 1.0                      |        |
| prev_se_clin_inf_pearl       | Probability PEARL-positive for 'clinical' and 'more infectious' category                | 1.0                      |        |
| prev_se_subclin_lowinf_plts  | Probability PLTS-positive for 'subclinical' and 'less infectious' category              | 0.3                      |        |
| prev_se_clin_lowinf_plts     | Probability PLTS-positive for 'clinical' and 'less infectious' category                 | 1.0                      |        |
| prev_se_subclin_inf_plts     | Probability PLTS-positive for 'subclinical' and 'more infectious' category              | 0.5                      |        |
| prev_se_clin_inf_plts        | Probability PLTS-positive for 'clinical' and 'more infectious' category                 | 1.0                      |        |
| prev_se_subclin_lowinf_cxr   | Probability CXR-positive for 'subclinical' and 'less infectious' category               | uniform (0.0, 0.5)       |        |
| prev_se_clin_lowinf_cxr      | Probability CXR-positive for 'clinical' and 'less infectious' category                  | 1.0                      |        |
| prev_se_subclin_inf_cxr      | Probability CXR-positive for 'subclinical' and 'more infectious' category               | uniform (0.5, 1.0)       |        |
| prev_se_clin_inf_cxr         | Probability CXR-positive for 'clinical' and 'more infectious' category                  | 1.0                      |        |
| prev_se_subclin_lowinf_ssx   | Probability SSx-positive for 'subclinical' and 'less infectious' category               | 0.0                      |        |
| prev_se_clin_lowinf_ssx      | Probability SSx-positive for 'clinical' and 'less infectious' category                  | 1.0                      |        |
| prev_se_subclin_inf_ssx      | Probability SSx-positive for 'subclinical' and 'more infectious' category               | 0.0                      |        |
| prev_se_clin_inf_ssx         | Probability SSx-positive for 'clinical' and 'more infectious' category                  | 1.0                      |        |
| tpt_completion_perc          | TPT completion percentage                                                               | 70.0                     | %      |
| rel_detection_subclin        | Relative detection rate of subclinical TB under passive case finding (ref. clinical TB) | 0.0                      |        |
| passive_detection_inflection | Time when passive detection started to scale up                                         | uniform (1990.0, 2020.0) |        |
| passive_detection_shape      | Shape parameter of passive detection scale-up profile                                   | uniform (0.05, 0.2)      |        |
| passive_detection_past_frac  | Past passive detection rate, as a fraction of the current one                           | uniform (0.5, 1.0)       |        |
| infection_pop_scale          | Exponent for population scaling in force of infection calculation                       | uniform (0.0, 1.0)       |        |

In [15]:
#| tab-params: "H" 
#| label: tab-priors
#| tab-cap: "Calibrated parameters"
params_file_path = DATA_FOLDER / "parameters.xlsx"
param_df = pd.read_excel(params_file_path, sheet_name="constant")
param_df["prior distribution"] = param_df.apply(
    lambda row: f"{row['distribution']} ({row['distri_param1']}, {row['distri_param2']})"
    if pd.notna(row['distribution'])
    else str(row['value']),
    axis=1
)
param_df = param_df.rename(columns={"full_text": "definition"})
param_df = param_df[param_df['distribution'].notna()]
param_df = param_df.fillna("")
md_table = param_df[["parameter", "definition", "prior distribution", "unit"]].to_markdown(index=False)  # index=False to skip the row numbers
display(Markdown(md_table))

| parameter                    | definition                                                                | prior distribution       | unit   |
|:-----------------------------|:--------------------------------------------------------------------------|:-------------------------|:-------|
| raw_transmission_rate        | Effective rate of transmission (before adjusting for infectiousness)      | uniform (1e-05, 0.001)   |        |
| rel_sus_contained            | Rel. risk of reinfection from 'contained' compartment (ref. 'mtb-naïve')  | uniform (0.2, 0.5)       |        |
| rel_sus_cleared              | Rel. risk of reinfection from 'cleared' compartment (ref. 'mtb-naïve')    | uniform (0.5, 1.0)       |        |
| breakdown_rate               | Rate of transition from 'contained' to 'incipient' (all ages)             | uniform (0.01, 1.0)      | /year  |
| clearance_rate               | Rate of transition from 'contained' to 'cleared' (all ages)               | uniform (0.01, 0.1)      | /year  |
| clinical_progression_rate    | Rate of progression from subclinical to clinical TB                       | uniform (0.5, 5.0)       | /year  |
| infectiousness_gain_rate     | Rate of progression from 'less infectious' to 'more infectious' TB        | uniform (0.5, 5.0)       | /year  |
| recent_detection_rate        | Annual rate of TB detection in 2020                                       | uniform (0.1, 5.0)       | /year  |
| prev_se_cleared              | Probability of TST positivity for 'cleared' category                      | uniform (0.2, 0.5)       |        |
| prev_se_subclin_lowinf_cxr   | Probability CXR-positive for 'subclinical' and 'less infectious' category | uniform (0.0, 0.5)       |        |
| prev_se_subclin_inf_cxr      | Probability CXR-positive for 'subclinical' and 'more infectious' category | uniform (0.5, 1.0)       |        |
| passive_detection_inflection | Time when passive detection started to scale up                           | uniform (1990.0, 2020.0) |        |
| passive_detection_shape      | Shape parameter of passive detection scale-up profile                     | uniform (0.05, 0.2)      |        |
| passive_detection_past_frac  | Past passive detection rate, as a fraction of the current one             | uniform (0.5, 1.0)       |        |
| infection_pop_scale          | Exponent for population scaling in force of infection calculation         | uniform (0.0, 1.0)       |        |

## Calibration targets

In [16]:

rows = []
for t in rt.targets:
    # each target contains a pandas Series with one value and one index
    year = t.data.index[0]
    value = t.data.values[0]
    
    descri = pl.title_lookup[t.name] if t.name in pl.title_lookup else ""

    if t.name == 'notifications':
        year = "1998-2022"
        value = "yr-dep"

    rows.append({
        "Target key": t.name,
        "Name": descri,
        "Year": year,
        "Value": value,
        "Likelihood": f"Normal, sd={round(t.stdev,2)}"
    })

df = pd.DataFrame(rows)

# Output as Markdown
display(Markdown(df.to_markdown(index=False)))

| Target key            | Name                            | Year      | Value   | Likelihood       |
|:----------------------|:--------------------------------|:----------|:--------|:-----------------|
| pearl_pos_per100k     | PEARL TB prevalence (/100k)     | 2024      | 900.4   | Normal, sd=91.88 |
| cxr_pos_per100k       | CXR TB prevalence (/100k)       | 2024      | 596.3   | Normal, sd=60.85 |
| tst_posXage_3_9_perc  | TST positivity 3-9yrs old (%)   | 2024      | 3.4     | Normal, sd=0.35  |
| tst_posXage_10_perc   | TST positivity 10-14yrs old (%) | 2024      | 9.4     | Normal, sd=0.96  |
| tst_posXage_15_perc   | TST positivity 15-64yrs old (%) | 2024      | 28.2    | Normal, sd=2.88  |
| tst_posXage_65_perc   | TST positivity 65+yrs old (%)   | 2024      | 32.7    | Normal, sd=3.34  |
| perc_prev_subclinical | % TB subclinical                | 2024      | 81.3    | Normal, sd=8.3   |
| perc_prev_infectious  | % TB more infectious            | 2024      | 69.5    | Normal, sd=7.09  |
| notifications         | TB notifications (n)            | 1998-2022 | yr-dep  | Normal, sd=40.48 |

In [17]:
from importlib import reload
reload(pl);

## Pairwise posterior distributions

In [18]:
#| fig-pos: "H" 
#| label: fig-pairs
#| fig-cap: "Pairwise relationships between calibrated parameters, with 2D kernel density estimates shown. The posterior mode is indicated by a black dot."
fig = pl.plot_posterior_pairs(idata, analysis_config['burn_in'], list(bcm.priors.keys()), 'kde')
display(fig)
plt.close(fig)

/Users/rrag0004/Documents/Code/tb_hierarchical/.pixi/envs/default/lib/python3.10/site-packages/arviz/plots/pairplot.py:224: UserWarning:

Divergences data not found, plotting without divergences. Make sure the sample method provides divergences data and that it is present in the `diverging` field of `sample_stats` or `sample_stats_prior` or set divergences=False



<Figure size 9000x8100 with 120 Axes>

\newpage
# Projected trajectories under the different screening scenarios

# TEMPORARILY REMOVED FIGURES FROM REPORT TO REDUCE LENGTH

In [19]:
# outputs_to_plot = [
#     'tb_incidence_per100k',
#     'viable_tbi_prevalence_perc',
#     'tb_prevalence_per100k',
#     'tb_mortality_per100k'
# ]

# for i_sc, sc in enumerate(rt.SCENARIOS):
#     if i_sc > 0:
#         display(Markdown("{{< pagebreak >}}"))
#     display(Markdown(f"## Scenario {sc.sc_name}"))
#     coverage = (
#         sc.scr_prgs[0].total_coverage_perc
#         if sc.scr_prgs else 0
#     )
#     display(Markdown(f"{sc.description}"))
#     display(Markdown(f"--> {round(coverage)}% coverage"))

#     fig, axes = plt.subplots(2, 2, figsize=(9, 7), sharex=False)
#     axes = axes.flatten()

#     for ax, output in zip(axes, outputs_to_plot):
#         out_name = (
#             pl.title_lookup[output]
#             if output in pl.title_lookup
#             else output
#         )

#         pl.plot_two_scenarios(
#             ax,
#             unc_dfs,
#             output,
#             scenarios=['baseline', sc.sc_id],
#             xlim=(2020, 2035),
#             include_unc=True,
#             ylab_fontsize=9
#         )

#         ax.set_title(out_name, fontsize=10)

#     # Optional: tidy layout
#     fig.tight_layout()
#     # display(fig)
#     plt.show()
#     plt.close(fig)

# Age-Specific Mixing Model

We model age-specific contact patterns using a socialising parameter framework, which allows for differences in mixing intensity across age groups in a simple and interpretable way. We assume differential levels of social activity for children (under 15 years), middle-age adults (15 to 64 years), and elderly (65 years and above). Each of these three groups is assigned a socialising parameter representing the relative opportunity of social contacts for individuals in that group. Adults are treated as the baseline with a socialising parameter of one, while the socialising parameters for children and elderly is calibrated to reflect different contact intensities.

The age-specific mixing matrix is constructed such that the contact intensity between two groups is proportional to the product of their respective socialising parameters. Within-group contact intensities are determined by squaring the socialising parameter of the corresponding age group, ensuring consistency with the same logic used for contacts between groups.

This framework captures weak age-assortative mixing, in that groups with lower socialising parameters tend to interact less with themselves and with other groups. Strong age-assortativity is not explicitly enforced but can emerge if non-adult groups have substantially lower socialising parameters. The approach is flexible, allowing the socialising parameters to be treated as calibration parameters, with uncertainty in these parameters propagating to the resulting mixing matrix. 

The resulting age-specific mixing matrix is then normalised such that it spectral radius is equal to one. This normalisation ensures that the overall contact intensity is appropriately scaled for use in the transmission model. It also reduces the correlation between the socialising parameters and the transmission parameter that scales the mixing matrix, facilitating more efficient model calibration.  